<a href="https://colab.research.google.com/github/Estebanarci2/Marketing-analitica3/blob/main/A)LIMPIEZA_Y_TRANSF_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U scikit-learn
!pip install sklearn.prepocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 44.3 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
ERROR: Could not find a version that satisfies the requirement sklearn.prepocessing (from versions: none)
ERROR: No matching distribution found for sklearn.prepocessing


In [ ]:
import numpy as np
import pandas as pd
import sqlite3 as sql
#import sklearn.prepocessing
import joblib
from ipywidgets import interact   ## analisis interactivo
#from sklearn import neighborns ## basado en contenido un solo


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## conectar la base de datos

conn = sql.connect ("/content/drive/MyDrive/analitica3/marketing /db_movies")
cur=conn.cursor()

In [ ]:
# consultar bases de datos que se pueden consultar por medio de sql
cur.execute("Select name from sqlite_master where type='table'")
cur.fetchall()

[('ratings',), ('movies',), ('usuarios_sel',), ('movies_sel',)]

In [ ]:
#Se asigna el valor de a la BD de ratings
a = "SELECT * FROM ratings"

In [ ]:
#Se consulta la BD de ratings
pd.read_sql_query(a, conn)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
#Se le asigna el valor b a la BD de movies
b = "SELECT * FROM movies"

In [ ]:
#Se consulta la BD de movies
pd.read_sql_query(b, conn)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [ ]:
#Se le asignan nombres a las consultas de SQL para llamarlas con sentencias de Pandas
ratings = pd.read_sql_query(a, conn)
movies = pd.read_sql_query(b, conn)

In [ ]:
ratings.describe()
# creo que deberiamos hacer el describe a la columna que se le va hacer al analisis por ejemplo a la rating para que no sea confuso

,userId,movieId,rating,timestamp
count,100836.000000,100836.000000,100836.000000,1.008360e+05
mean,326.127564,19435.295718,3.501557,1.205946e+09
std,182.618491,35530.987199,1.042529,2.162610e+08
min,1.000000,1.000000,0.500000,8.281246e+08
25%,177.000000,1199.000000,3.000000,1.019124e+09
50%,325.000000,2991.000000,3.500000,1.186087e+09
75%,477.000000,8122.000000,4.000000,1.435994e+09
max,610.000000,193609.000000,5.000000,1.537799e+09


**(POSIBLE ANÁLISIS) Se observa que la desviación estándar de las películas que han visto los usuarios es demasiado alta pero esto puede llegar a ser un error debido a que se está calculando un valor que no es de naturaleza entera sino que debería ser categórica debido a que representa una identificación única de la película**

In [ ]:
movies.describe()

,movieId
count,9742.000000
mean,42200.353623
std,52160.494854
min,1.000000
25%,3248.250000
50%,7300.000000
75%,76232.000000
max,193609.000000


**(POSIBLE ANÁLISIS) Se hace una consulta para observar un resumen estadístico de la BD donde se encuentra que la desviación estándar está por encima de la media por lo que se concluiría que hay un gran sesgo en la información pero en este caso primero hay que observar la naturaleza de las variables debido a que se pueden estar haciendo cálculos con variables que son de naturaleza categórica**

In [ ]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


**(POSIBLE ANÁLISIS) Efectivamente los ID de los usuarios se están tomando como números para realizar cálculos por lo que se procede a transformarlos a tipo object**

In [ ]:
#Cambio de columnas categóricas a tipo Object

#ratings[['userId','movieId']] = ratings[['userId','movieId']].astype('object')
#movies['movieId'] = ratings['movieId'].astype('object')

In [ ]:
#Consulta para corroborar los cambios

#ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  object 
 1   movieId    100836 non-null  object 
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 3.1+ MB


In [ ]:
#Consulta para corroborar los cambios

#movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   object
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: object(3)
memory usage: 228.5+ KB


**(POSIBLE ANÁLISIS) Luego de cambiar las variables categóricas a tipo Object se repite el resumen estadístico**

In [ ]:
#ratings.describe()

,rating,timestamp
count,100836.000000,1.008360e+05
mean,3.501557,1.205946e+09
std,1.042529,2.162610e+08
min,0.500000,8.281246e+08
25%,3.000000,1.019124e+09
50%,3.500000,1.186087e+09
75%,4.000000,1.435994e+09
max,5.000000,1.537799e+09


In [ ]:
#movies.describe()

,movieId,title,genres
count,9742,9742,9742
unique,3175,9737,951
top,356,Emma (1996),Drama
freq,38,2,1053


In [ ]:
#Se consulta si hay películas duplicadas
movies['movieId'].duplicated().sum()

0

In [ ]:
#Se consulta si hay usuarios duplicados
ratings['userId'].duplicated().sum()

100226

**La mayoría de usuarios en la BD de ratings están duplicados pero no significa que haya errores debido a que cada registro corresponde a una película que vio el usuario por lo que no hay necesidad de imputar**

In [ ]:
#Se crea una nueva tabla que filtra cuántas películas vio cada usuario
#discriminando sólo a los que han visto entre 10 y 600

tabla_existente = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table' AND name='usuarios_sel'", conn)

# Si la tabla usuarios_sel no existe, entonces la creamos
if tabla_existente.empty:
    consulta_sql = """
        CREATE TABLE usuarios_sel AS
        SELECT userId, COUNT(*) AS n_id
        FROM ratings
        GROUP BY userId
        HAVING n_id >= 10 AND n_id <= 600
        ORDER BY n_id ASC
    """

    pd.read_sql_query(consulta_sql, conn)
else:
    print("La tabla usuarios_sel ya existe en la base de datos.")

La tabla usuarios_sel ya existe en la base de datos.


In [ ]:
#Se consulta la tabla del total de películas que ha visto cada usuario entre 10 y 600
pd.read_sql_query("SELECT * FROM usuarios_sel", conn)

,userId,n_id
0,53,20
1,147,20
2,189,20
3,194,20
4,207,20
...,...,...
573,219,528
574,28,570
575,91,575
576,555,578


**Es preciso excluír de la tabla los usuarios que han visto menos de 10 películas porque quizá no resulte muy confiable su calificación. También se decide excluír a las personas que han visto más de 600 para tener una cifra un poco más redonda ya que el máximo de la BD es 610, además de que la frecuencia con que una persona puede haber visto tantas películas es poco común y quizá pueda sesgar los datos**

In [ ]:
#Se crea una nueva tabla que filtra cuántas veces ha sido vista cada película
#discriminando sólo a las que se han visto mínimo 50 veces

# Eliminamos la tabla si existe
cur.execute("DROP TABLE IF EXISTS movies_sel")

# Creamos la tabla movies_sel
cur.execute("""
    CREATE TABLE movies_sel AS
    SELECT movieId, COUNT(*) AS n_movie
    FROM ratings
    GROUP BY movieId
    HAVING n_movie >= 50
    ORDER BY n_movie DESC
""")

In [ ]:
pd.read_sql_query("SELECT * FROM movies_sel", conn)

,movieId,n_movie
0,356,329
1,318,317
2,296,307
3,593,279
4,2571,278
...,...,...
445,3785,50
446,2424,50
447,2105,50
448,910,50


**Se decide excluír de la tabla las películas que se hayan visto menos de 50 veces debido a que se encontró que hay demasiadas con pocas visualizaciones y puede que no sean una buena opción para recomendar a los usuarios**

In [ ]:
#Se guardan las tablas filtradas en variables para consultarlas con sentencias de Pandas

userviews = pd.read_sql_query("SELECT * FROM usuarios_sel", conn)
movieviews = pd.read_sql_query("SELECT * FROM movies_sel", conn)